In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [2]:
# Step 1: Load the dataset safely
try:
    df = pd.read_csv(r"C:\Users\K.Pushpavali\Downloads\medicine_dataset_100.csv")
except FileNotFoundError:
    print("ERROR: medicine_dataset_100.csv not found. Make sure it is in the working directory.")
    df = pd.DataFrame(columns=["Medicine", "Quantity", "Age", "Purpose"])

# Normalize column names and medicine names
df.columns = [col.strip().capitalize() for col in df.columns]
if 'Medicine' in df.columns:
    df['Medicine'] = df['Medicine'].str.lower()
else:
    print("WARNING: 'Medicine' column missing in dataset.")

In [3]:
# Step 2: Define intents
intents = [
    {
        "tag": "greeting",
        "patterns": ["hi", "hello", "hey", "good morning", "good afternoon"],
        "response": "Hello! How can I assist you today?"
    },
    {
        "tag": "thanks",
        "patterns": ["thanks", "thank you", "thx", "appreciate it"],
        "response": "You're welcome!"
    },
    {
        "tag": "goodbye",
        "patterns": ["bye", "goodbye", "see you later", "exit"],
        "response": "Goodbye! Take care and stay healthy."
    },
    {
        "tag": "help",
        "patterns": ["i need help", "can you help me", "how does this work", "what can you do"],
        "response": "I can provide details about medicines like quantity, age group, and usage. Just ask me about any medicine!"
    },
    {
        "tag": "symptoms",
        "patterns": ["what are the symptoms of flu", "signs of cold", "how to know if i have fever"],
        "response": "Common symptoms include headache, fever, cough, body ache, or fatigue. Please consult a doctor for a proper diagnosis."
    },
    {
        "tag": "doctor_check",
        "patterns": ["are you a doctor", "can you prescribe", "can you treat me"],
        "response": "I'm not a doctor, but I can give general information about medicines. Please consult a licensed physician for prescriptions."
    },
    {
        "tag": "side_effects",
        "patterns": ["what are the side effects", "is this medicine safe", "can this cause problems"],
        "response": "Medicines can have side effects like nausea, dizziness, or allergies. Check the leaflet or consult a pharmacist."
    },
    {
        "tag": "not_feeling_well",
        "patterns": ["i feel sick", "i don't feel good", "i'm unwell", "i'm feeling dizzy"],
        "response": "I'm sorry to hear that. Please take rest and consult a doctor if symptoms persist."
    },
    {
        "tag": "medicine_check",
        "patterns": ["can i take paracetamol", "is ibuprofen safe", "should i use aspirin"],
        "response": "Let me check that for you. Please tell me your age and symptoms so I can give better info."
    }
]

In [4]:
# Step 3: Prepare training data
corpus = []
labels = []

for intent in intents:
    for pattern in intent["patterns"]:
        corpus.append(pattern.lower())
        labels.append(intent["tag"])

In [5]:
# Step 4: Train model
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
model = MultinomialNB()
model.fit(X, labels)

MultinomialNB()

In [6]:
# Step 5: Prediction functions
def predict_intent(text):
    text = text.lower()
    input_data = vectorizer.transform([text])
    predicted = model.predict(input_data)[0]

    for intent in intents:
        if intent["tag"] == predicted:
            return intent["response"]
    return "I'm not sure I understand. Can you rephrase?"

def medical_chatbot(user_input):
    user_input_lower = user_input.lower()

    # Check medicine info
    if not df.empty:
        for _, row in df.iterrows():
            if row['Medicine'] in user_input_lower:
                return (
                    f"Medicine: {row['Medicine'].title()}\n"
                    f"Quantity: {row['Quantity']}\n"
                    f"Age: {row['Age']}\n"
                    f"Purpose: {row['Purpose']}"
                )
    
    # If no medicine found, respond by intent
    return predict_intent(user_input)

In [ ]:
def chat_with_bot():
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Bot: Goodbye!")
            break
        response = medical_chatbot(user_input)
        print("Bot:", response)

chat_with_bot()

You: hello
Bot: Hello! How can I assist you today?
You: hi
Bot: Hello! How can I assist you today?
You: Lisinopril
Bot: Medicine: Lisinopril
Quantity: 5-40 mg tablet
Age: Adults
Purpose: Treats high blood pressure and heart failure.
You: Paracetamol
Bot: Medicine: Paracetamol
Quantity: 500 mg tablet
Age: Adults and children above 12 years
Purpose: Relieves mild to moderate pain and reduces fever.
You: Thanks
Bot: You're welcome!
